In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [2]:
# Definir la región de interés
roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/MalawiWoodfuelSite_1a')
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
# Añadir Google Satellite como mapa base
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

In [3]:
roi_feature = ee.FeatureCollection(roi).first()

# Extraer la geometría de la característica
roi_geometry = roi_feature.geometry()

# Obtener las coordenadas de la geometría
roi_coordinates = roi_geometry.coordinates().getInfo()

# Imprimir las coordenadas para verificar
print("Coordenadas de ROI:", roi_coordinates)

# Guardar las coordenadas en una variable
roi_coords = roi_coordinates

ee_roi =  ee.Geometry.Polygon(roi_coords) 

Coordenadas de ROI: [[[35.147307857226416, -15.036316131901854], [35.15253608286859, -15.223952919778808], [35.15776440008984, -15.411589648079637], [35.20994926360607, -15.410686693842585], [35.26213417872549, -15.409783744326894], [35.314318994087046, -15.408880778443212], [35.366503893152895, -15.407977790870213], [35.3589679641149, -15.032891601485606], [35.30605294737285, -15.033747715996299], [35.25313786054075, -15.034603909691883], [35.20022286170109, -15.035460051719076], [35.147307857226416, -15.036316131901854]]]


In [32]:
# Definir el año de análisis
year = 2010

# Definir la fecha de inicio y fin para filtrar la colección
start = f'{year}-01-01'
end = f'{year}-12-31'

In [33]:
# Cargar la colección de Landsat 7
l7_col = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10))

def mask_clouds_landsat(image):
    # Seleccionar la banda de máscara de calidad (QA_PIXEL)
    qa = image.select('QA_PIXEL')

    # Crear una máscara para las nubes y sombras de nubes
    cloud_shadow_bit_mask = ee.Number(2).pow(3).int()
    clouds_bit_mask = ee.Number(2).pow(5).int()

    # Aplica las máscaras de nubes y sombras
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0) \
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))

    # Aplicar la máscara de nubes y sombras
    return image.updateMask(mask)

# Aplicar la máscara de nubes a la colección de Landsat 7
l7_masked = l7_col.map(mask_clouds_landsat)
image = l7_masked.median().toFloat().clip(roi)

# Calcular índices
ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
mndwi = image.normalizedDifference(['SR_B2', 'SR_B5']).rename('MNDWI')
ndbi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDBI')

evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': image.select('SR_B4'),
        'RED': image.select('SR_B3'),
        'BLUE': image.select('SR_B1')
    }
).rename('EVI')

savi = image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {
        'NIR': image.select('SR_B4'),
        'RED': image.select('SR_B3')
    }
).rename('SAVI')

image = image.addBands([ndvi, mndwi, ndbi, evi, savi])

# Cargar la colección Dynamic World y filtrar por la grilla y la fecha
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-12-01')

# Añadir la banda de etiqueta de Dynamic World a la imagen
dw_label = dw_col.select('label').median().clip(roi)
image = image.addBands(dw_label.rename('label'))

# Añadir DEM y pendiente
dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)

image = image.addBands(dem.rename('elevation'))
image = image.addBands(slope)

# Funciones de preprocesamiento
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

def radar_to_db_palsar(image, band):
    """
    Convierte los datos de la banda HH y HV de PALSAR según la fórmula dada.
    Machine learning feature importance selection for predicting aboveground biomass in African savannah with landsat 8 and ALOS PALSAR data Sa'ad Ibrahim
    """
    dn_squared = ee.Image(image).select(band).pow(2)
    backscatter = ee.Image(10).multiply(dn_squared.log10()).subtract(83.01).rename(band + '_dB')
    return backscatter

def preprocess(image):
    speckle_filtered = image.focal_median(kernelType='circle', radius=50, units='meters')
    return speckle_filtered


# Procesamiento de imágenes PALSAR
img_hh = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HH')
    .map(preprocess)
)

img_hv = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH')
    .filterBounds(roi)
    .filterDate(start, end)
    .select('HV')
    .map(preprocess)
)

# Aplicar la conversión a dB
palsar_hh_db = img_hh.map(lambda img: radar_to_db_palsar(img, 'HH'))
palsar_hv_db = img_hv.map(lambda img: radar_to_db_palsar(img, 'HV'))

# Calcular la mediana
palsar_hh_median = palsar_hh_db.median().rename('HH_dB')
palsar_hv_median = palsar_hv_db.median().rename('HV_dB')

# Calcular los índices adicionales
hh_plus_hv = palsar_hh_median.add(palsar_hv_median).rename('HH_plus_HV')
hh_minus_hv = palsar_hh_median.subtract(palsar_hv_median).rename('HH_minus_HV')
hh_div_hv = palsar_hh_median.divide(palsar_hv_median).rename('HH_div_HV')

# Calcular el índice RVI
rvi = palsar_hv_median.divide(palsar_hh_median.add(palsar_hv_median)).multiply(4).rename('RVI_palsar')

# Combinar todas las bandas en una sola imagen
palsar_combined = ee.Image.cat([palsar_hh_median, palsar_hv_median, hh_plus_hv, hh_minus_hv, hh_div_hv, rvi]).clip(roi)

# Añadir las bandas combinadas a la imagen original
image = image.addBands(palsar_combined)




In [34]:
def calculate_texture_metrics(image, index_name, roi, scale_factor=100, size=5):
    # Normalizar el índice y convertirlo a tipo int32
    index_norm = image.select(index_name).multiply(scale_factor).add(scale_factor).toInt32().clip(roi)
    
    # Calcular texturas usando el índice normalizado
    glcm = index_norm.glcmTexture(size=size)
    
    # Extraer las texturas de interés y renombrarlas
    contrast = glcm.select(f'{index_name}_contrast').rename(f'{index_name}_Contrast')
    correlation = glcm.select(f'{index_name}_corr').rename(f'{index_name}_Correlation')
    entropy = glcm.select(f'{index_name}_ent').rename(f'{index_name}_Entropy')
    inertia = glcm.select(f'{index_name}_inertia').rename(f'{index_name}_Inertia')
    
    # Agregar las bandas de textura a la imagen original
    return image.addBands([contrast, correlation, entropy, inertia])

# Aplicar la función a NDVI, SAVI y EVI
image = calculate_texture_metrics(image, 'NDVI', roi)
image = calculate_texture_metrics(image, 'SAVI', roi)
image = calculate_texture_metrics(image, 'EVI', roi)


def calculate_texture_metrics(image, band_name, roi, size=5):
    # Convertir la banda a tipo int32 (no se realiza normalización adicional)
    band_int = image.select(band_name).toInt32().clip(roi)
    
    # Calcular texturas usando la banda convertida a int32
    glcm = band_int.glcmTexture(size=size)
    
    # Extraer las texturas de interés y renombrarlas
    contrast = glcm.select(f'{band_name}_contrast').rename(f'{band_name}_Contrast')
    correlation = glcm.select(f'{band_name}_corr').rename(f'{band_name}_Correlation')
    entropy = glcm.select(f'{band_name}_ent').rename(f'{band_name}_Entropy')
    inertia = glcm.select(f'{band_name}_inertia').rename(f'{band_name}_Inertia')
    
    # Agregar las bandas de textura a la imagen original
    return image.addBands([contrast, correlation, entropy, inertia])

# Aplicar la función a las bandas HH_dB y HV_dB
image = calculate_texture_metrics(image, 'HH_dB', roi)
image = calculate_texture_metrics(image, 'HV_dB', roi)

bands =  [ 'NDVI_Contrast', 'NDVI_Correlation', 'NDVI_Entropy', 'NDVI_Inertia','EVI', 'EVI_Inertia', 'HH_dB', 'HH_dB_Contrast', 'HH_dB_Correlation', 'HH_dB_Entropy', 'HH_dB_Inertia', 'HH_div_HV', 'HH_minus_HV', 'HH_plus_HV', 'HV_dB', 'HV_dB_Contrast', 'HV_dB_Correlation', 'HV_dB_Entropy', 'HV_dB_Inertia', 'MNDWI', 'NDBI', 'NDVI',  'RVI_palsar', 'SAVI', 'SAVI_Contrast', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'elevation', 'slope']



image = image.select(bands)

In [35]:
feature_names =  [ 'NDVI_Contrast', 'NDVI_Correlation', 'NDVI_Entropy', 'NDVI_Inertia','EVI', 'EVI_Inertia', 'HH_dB', 'HH_dB_Contrast', 'HH_dB_Correlation', 'HH_dB_Entropy', 'HH_dB_Inertia', 'HH_div_HV', 'HH_minus_HV', 'HH_plus_HV', 'HV_dB', 'HV_dB_Contrast', 'HV_dB_Correlation', 'HV_dB_Entropy', 'HV_dB_Inertia', 'MNDWI', 'NDBI', 'NDVI',  'RVI_palsar', 'SAVI', 'SAVI_Contrast', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'elevation', 'slope']
 


label = "agbd"

user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/Rf_Malawi-texture4"
asset_id

# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)

In [11]:
feature_names = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'MNDWI', 'NDBI', 'NDVI', 'EVI', 'SAVI', 'elevation', 'slope', 
         'HH_dB', 'HV_dB', 'HH_plus_HV', 'HH_minus_HV', 'HH_div_HV', 'RVI_palsar']  
label = "agbd"
user_id = 'users/facuboladerasgee'
def recombine_model_parts(user_id, base_asset_id, num_parts):
    parts = [ee.FeatureCollection(f"{base_asset_id}_part_{i}") for i in range(num_parts)]
    combined_fc = ee.FeatureCollection(parts[0])
    
    for part in parts[1:]:
        combined_fc = combined_fc.merge(part)
    
    return ml.fc_to_classifier(combined_fc)

# Ejemplo de uso:
num_parts = 2  # Ajusta este número según la cantidad de partes exportadas
base_asset_id = f"{user_id}/Rf_Malawi-L-P"
recombined_classifier = recombine_model_parts(user_id, base_asset_id, num_parts)

# Clasificar la imagen con el clasificador recombinado
classified = image.select(feature_names).classify(recombined_classifier)


In [14]:
# # Reducir la imagen clasificada a su valor máximo en la región de interés
# max_value = classified.reduceRegion(
#     reducer=ee.Reducer.max(),
#     geometry=roi,
#     scale=100,
#     maxPixels=1e9
# )

# # Obtener el valor máximo de la reducción
# max_agbd = max_value.get('classification').getInfo()
# print(f"Max value of the classified image: {max_agbd}")

Max value of the classified image: 155.93490694999983


In [36]:
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
Map.centerObject(roi)

visPredictedBiomass = {
    'min': 0,
    'max': 200,
    'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
}

Map = geemap.Map(zoom=11)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

Map.addLayer(
    classified,
    visPredictedBiomass,
    "classification",)
Map.add_colorbar(visPredictedBiomass, label="Predicted Biomass")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [37]:
import time

# Parámetros de exportación
task = ee.batch.Export.image.toDrive(
    image=classified,
    description='Malawi_2010-texture2',
    folder='Estimators_Malawi_complete',
    region=ee_roi,
    scale=30,
    crs='EPSG:4326',  # CRS adecuado para Nepal
    
    
)
task.start()

print("Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.")

# Esperar a que la tarea de exportación se complete (opcional)
while task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente

if task.status()['state'] == 'COMPLETED':
    print('Exportación completada con éxito.')
else:
    print('Error en la exportación:', task.status())

Exportación iniciada. Revisa tu Google Drive en la carpeta 'EE_RF'.
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación en progreso...
Exportación completada con éxito.
